In [ ]:
# Instalando as bibliotecas para o web scraping

%pip install selenium
%pip install beautifulsoup4

In [1]:
# Importando as Bibliotecas

from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from multiprocessing.dummy import Pool as ThreadPool
import time

In [9]:
def pegar_conteudo(url):
    dados_curso = [url.split('/')[4]]

    driver = webdriver.Firefox(options=opcoes)
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CourseAbout__header__title')) and EC.presence_of_element_located((By.CLASS_NAME, 'CourseAbout__header__info')))

        conteudo = BeautifulSoup(driver.page_source, 'html.parser')    
        driver.quit()
        try:
            nome_curso = conteudo.find('h1', class_='CourseAbout__header__title').text
        except:
            nome_curso = ''

        if len(nome_curso) == 0:
            nome_curso = 'Nome não disponível'
        dados_curso.append(nome_curso)

        try:
            num_alunos = int(conteudo.find('div', class_='CourseAbout__header__info CourseAbout__header__info--students').text[:-7])
        except:
            num_alunos = 'Número de alunos não disponível'
        dados_curso.append(num_alunos)

        return dados_curso
    except:
        driver.quit()
        return None

def pegar_dados(curso_max):

    base_url = 'https://app.sapium.com.br/course/{}/about'
    urls = [base_url.format(i) for i in range(1, curso_max)]

    global opcoes
    opcoes = webdriver.FirefoxOptions()

    pool = ThreadPool(8)
    dados_cursos = pool.map(pegar_conteudo, urls)
    pool.close()
    pool.join()

    dados_cursos = [dado for dado in dados_cursos if dado]

    cursos_df = pd.DataFrame(dados_cursos, columns=['Número', 'Nome', 'Número de Alunos'])
    cursos_df = cursos_df.set_index('Número')

    return cursos_df

cursos_df = pegar_dados(1000)
cursos_df

,Nome,Número de Alunos
Número,,
1,Curso gratuito de Marketing Digital básico,390
2,Nome não disponível,127
3,Curso gratuito de Photoshop CC do zero ao avan...,146
4,Curso de Indesign CC Completo,122
5,Curso gratuito de Direito Constitucional para OAB,49
...,...,...
888,Curso Online Mandala Orquídea 12 Pontas,Número de alunos não disponível
897,Master Class Presença Digital,43
928,Nome não disponível,576


In [21]:
for i in range(1, len(cursos_df["Número de Alunos"])):
    try:
        if cursos_df["Número de Alunos"][i] > 1000:
            print(cursos_df.iloc[i])
    except:
        pass

Nome                Nome não disponível
Número de Alunos                   7234
Name: 655, dtype: object
Nome                Transformando a sua Vida Financeira
Número de Alunos                                   5818
Name: 731, dtype: object
